In [1]:
%load_ext autoreload
%autoreload 2

Here I import the Dataset and Subject classes.

In [2]:
from cnbpy.datalad import Dataset,Subject

I then define the dataset to be downloaded to a location that should also be readable to you.

In [3]:
mydata=Dataset(local_base='/storage/research/cinn_comp/cnbpy_Resources/datalad_test',source='///abide/RawDataBIDS/Caltech')

I then 'install' it. Recall that this doesnt actually download anything just yet.

In [4]:
mydata.install()

In [5]:
from cnbpy.bids import BIDS

mybids=BIDS(mydata.dset.path)

I now use the BIDS class to see what subjects are available.

In [ ]:
mybids.subjects


Loop through all of these subjects and run fMRIprep - super hacky!

In [ ]:
import nibabel as nib
from cnbpy.fmriprep import FMRIPREP

# Loop through all subjects in the list and run fMRI prep
#set count to 0 (corresponding to the first subject in the list)
count = 0
#for every subject in mybids.subjects, do:
for s in mybids.subjects:
    #create a variable called mysub that tells us where the subject data are located
    mysub=Subject(mydata,subid=mybids.subjects[count])
    #print the subject ID
    print("Working on sub-", mybids.subjects[count])
    #Retrieve that subject's data from datalad
    mysub.get()
    #get the anatomical file
    anat=mybids.find_anats(sub=s)
    #load the anatomical file using nibabel
    nm=nib.load(anat[-1])
    #get the rest of the data for that subject?
    dat=nm.get_data()
    #get the tsv file for that participant
    mydata.get_info_files()
    
    #set up frmiprep job
    myfmriprep=FMRIPREP(jobname='Carolyn_job',pid=mybids.subjects[count])
    #create the SLURM job file and print it so I can see
    myfmriprep.populate_all()
    print(myfmriprep.working_string)
    
    #send the job to the cluster
    #myfmriprep.writeout()
    #myfmriprep.execute()
    
    #add 1 to the count to move to the next participant
    count = count + 1


Now we can remove the data

In [16]:
mydata.remove()